# Симплекс-метод

In [1]:
import numpy as np
from symplexmethod import symplex, main, initial, dual
from transportproblem import transport_problem

ROUND_DEC_POINTS = 6 # for rounding

In [2]:
# the first example
c0 = np.array([1, 1, 0, 0, 0])
A0 = np.array([[-1, 1, 1, 0, 0],
              [1, 0, 0, 1, 0],
              [0, 1, 0, 0, 1]])
b0 = np.array([1, 3, 2])
x0 = np.array([0, 0, 1, 3, 2])
B0 = np.array([2, 3, 4])

# TODO: more tests

tests1 = [
    {
        'c': c0,
        'A': A0,
        'x': x0,
        'b': b0,
        'B': B0,
    },
]

### Тестирование основной фазы симплекс-метода

Сравнение результатов вызова полного симплекс-метода, реализованного в `scipy`, принимающего параметры ЗЛП 1: `c`, `A` и `b` (каноническая форма), с результатами вызова моего алгоритма основной фазы симплекс-метода для исходных данных `c`, `A`, `x` (начальный базисный допустимый план) и `B` (вектор базисных индексов), соответствующих ЗЛП 1.

In [3]:
for i, test in enumerate(tests1):
    print(f'TEST #{i+1}:')
    print('\tSCIPY:')

    try:
        solution = symplex.scipy_solve(test['c'], test['A'], test['b'])
        x = [round(x, ROUND_DEC_POINTS) for x in solution.x]
        print(f'\t\tx = {x}')
    except:
        print('\t\t[ERROR]')

    print('\tCUSTOM:')
    try:
        solution = main.run(test['c'], test['A'], test['x'], test['B'])
        if solution['solved']:
            if solution['unbound']:
                print('\t\t- unbound')
            else:
                print('\t\t- bound')
                x = solution['x']
                print(f'\t\t- x = {x}')
        else:
            details = solution.get('details')
            print('\t\t[FAILURE]; details:')
            if details:
                for line in details:
                    print(f'\t\t\t{line}')
            else:
                print('\t\t\t(no details)')
    except Exception as e:
        print(f'\t\t[ERROR]: {e}')
    print()

TEST #1:
	SCIPY:
		x = [3.0, 2.0, 2.0, 0.0, 0.0]
	CUSTOM:
		- bound
		- x = [3 2 2 0 0]



### Тестирование начальной фазы симплекс-метода

In [4]:
tests2 = [
    {
        'c': [1, 0, 0],
        'A': [[-1, -1, -1], [-2, -2, -2]],
        'b': [-10, -20]
    },
    {
        'c': c0,
        'A': A0,
        'b': b0
    },
    {
        'c': [-1, 1, 0],
        'A': [[2, 1, 3],
              [1, -3, 1],
              [1, 11, 3]],
        'b': [1, -3, 11]
    },
    {
        'c': '',
        'A': [[-4, -3, -2, 1, 0, 0],
              [-3, -2, -1, 0, 1, 0],
              [-1, -1, -2, 0, 0, 1]],
        'b': [-33, -23, -12]
    }
]

In [5]:
for i, test in enumerate(tests2):
    print(f'TEST #{i+1}:')

    try:
        solution = initial.run(test['c'], test['A'], test['b'])
        if solution['success']:
            if solution['infeasible']:
                print('\t- infeasible')
            else:
                print('\t- feasible')
                x = solution['x']
                print(f'\t- x = {x}')
                B = solution['B']
                print(f'\t- B = {B}')
                A = solution['A']
                print('\t- A:')
                for row in A:
                    print(f'\t\t{row}')
                b = solution['b']
                print(f'\t- b = {b}')
        else:
            details = solution.get('details')
            print('\t[FAILURE]; details:')
            if details:
                for line in details:
                    print(f'\t\t{line}')
            else:
                print('\t\t(no details)')
    except Exception as e:
        print(f'\t\t[ERROR]: {e}')

TEST #1:
	- feasible
	- x = [10.  0.  0.]
	- B = [0]
	- A:
		[1 1 1]
	- b = [10]
TEST #2:
	- feasible
	- x = [3. 2. 2. 0. 0.]
	- B = [1, 2, 0]
	- A:
		[-1  1  1  0  0]
		[1 0 0 1 0]
		[0 1 0 0 1]
	- b = [1 3 2]
TEST #3:
	- feasible
	- x = [0. 1. 0.]
	- B = [1 0]
	- A:
		[2 1 3]
		[-1  3 -1]
	- b = [1 3]
TEST #4:
	- feasible
	- x = [5. 3. 2. 0. 0. 0.]
	- B = [1, 0, 2]
	- A:
		[4, 3, 2, -1, 0, 0]
		[3, 2, 1, 0, -1, 0]
		[1, 1, 2, 0, 0, -1]
	- b = [33, 23, 12]


### Тестирование двойственого симплекс-метода

In [12]:
tests3 = [
    {
        'c': np.array([-4, -3, -7, 0, 0]),
        'A': np.array([[-2, -1, -4, 1, 0],
                       [-2, -2, -2, 0, 1]]),
        'b': np.array([-1, -1.5]),
        'B': np.array([3, 4])
    },
    # the following tests by https://github.com/UnstoppableGuy :
    {
        'c': np.array([2, 2, 1, -10, 1, 4, -2, -3]),
        'A': np.array([[-2, -1, 1, -7, 0, 0, 0, 2],
                       [4, 2, 1, 0, 1, 5, -1, -5],
                       [1, 1, 0, -1, 0, 3, -1, 1]]),
        'b': np.array([-2, 4, 3]),
        'B': np.array([1, 4, 6])
    },
    {
        'c': np.array([12, -2, -6, 20, -18, -5, -7, -20]),
        'A': np.array([[-2, -1, 1, -7, 0, 0, 0, 2],
                       [-4, 2, 1, 0, 1, 5, -1, 5],
                       [1, 1, 0, 1, 4, 3, 1, 1]]),
        'b': np.array([-2, 8, -2]),
        'B': np.array([1, 3, 5])
    },
    {
        'c': np.array([10, -2, -38, 16, -9, -9, -5, -7]),
        'A': np.array([[-2, -1, 10, -7, 1, 0, 0, 2],
                       [-4, 2, 3, 0, 5, 1, -1, 0],
                       [1, 1, 0, 1, -4, 3, -1, 1]]),
        'b': np.array([-2, -5, 2]),
        'B': np.array([1, 7, 4])
    }
]

In [13]:
for i, test in enumerate(tests3):
    print(f'TEST #{i+1}:')

    try:
        solution = dual.run(test['c'], test['A'], test['b'], test['B'])
        scipy_solution = symplex.scipy_solve(test['c'], test['A'], test['b'])
        if solution['solved']:
            if solution['infeasible']:
                print('\t- infeasible')
                print(f'\t- scipy: {scipy_solution.message}')
            else:
                print('\t- feasible')
                kappa = [round (x, ROUND_DEC_POINTS) for x in solution['kappa']]
                print(f'\t- kappa = {kappa}')
                scipy_kappa = [round(x, ROUND_DEC_POINTS) for x in scipy_solution.x]
                print(f'\t- kappa = {scipy_kappa} (scipy)')
                y = solution['y']
                print(f'\t- y = {y}')
                B = solution['B']
                print(f'\t- B = {B}')
        else:
            print('\t[FAILURE]')
    except Exception as e:
        print(f'\t\t[ERROR]: {e}')

TEST #1:
	- feasible
	- kappa = [0.25, 0.5, 0.0, 0.0, 0.0]
	- kappa = [0.25, 0.5, 0.0, 0.0, 0.0] (scipy)
	- y = [1. 1.]
	- B = [1 0]
TEST #2:
	- feasible
	- kappa = [0.0, 2.666667, 0.0, 0.0, 0.333333, 0.0, 0.0, 0.333333]
	- kappa = [0.0, 2.666667, 0.0, 0.0, 0.333333, 0.0, 0.0, 0.333333] (scipy)
	- y = [0.66666667 1.         0.66666667]
	- B = [1 7 4]
TEST #3:
	- infeasible
	- scipy: The algorithm terminated successfully and determined that the problem is infeasible.
TEST #4:
	- feasible
	- kappa = [1.35, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.45]
	- kappa = [1.35, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.45] (scipy)
	- y = [-2.7  -1.55 -1.6 ]
	- B = [1 7 0]


### Тестирование решения транспортной задачи

In [8]:
tests4 = [
    {
        'a': np.array([100, 300, 300]),
        'b': np.array([300, 200, 200]),
        'C': np.array([[8, 4, 1],
                       [8, 4, 3],
                       [9, 7, 5]])
    },
    
]

In [9]:
for i, test in enumerate(tests4):
    print(f'TEST #{i+1}:')

    try:
        X, B = transport_problem.nw_corner_method(test['a'], test['b'])
        print(f'\t- X:')
        print('\t\t', end='')
        print('\n\t\t'.join([str(line) for line in X]))
        print(f'\t- B = {B}')
    except Exception as e:
        print(f'\t\t[ERROR]: {e}')

TEST #1:
	- X:
		[100.   0.   0.]
		[200. 100.   0.]
		[  0. 100. 200.]
	- B = [(0, 0), (1, 0), (1, 1), (2, 1), (2, 2)]
